In [ ]:
#r "nuget:Microsoft.Diagnostics.NETCore.Client,0.2.61701"
using Microsoft.Diagnostics.NETCore.Client;
using System.Diagnostics;
using System.Linq;

var availableProcs = DiagnosticsClient.GetPublishedProcesses()
    .Select(x => Process.GetProcessById(x))
    .Select(p => new { name = p.ProcessName, id = p.Id, module = p.MainModule.FileName });

foreach (var p in availableProcs)
    display(p);

In [ ]:
#r "nuget:Microsoft.Diagnostics.Tracing.TraceEvent,2.0.49"
#r "nuget:Microsoft.Diagnostics.NETCore.Client,0.2.61701"

using Microsoft.Diagnostics.NETCore.Client;
using System.Diagnostics;
using Microsoft.Diagnostics.Tracing;
using Microsoft.Diagnostics.Tracing.Parsers;
using System.Threading.Tasks;

var client = new DiagnosticsClient(availableProcs.Where(p => p.name == "ConsoleApp").FirstOrDefault()?.id ?? -1);
var session = client.StartEventPipeSession(
    providers: new List<EventPipeProvider> 
    { 
        new EventPipeProvider("Microsoft-Windows-DotNETRuntime", System.Diagnostics.Tracing.EventLevel.Verbose, (long)(-1))
    }
);

var source = new EventPipeEventSource(session.EventStream);
source.Clr.All += (TraceEvent obj) =>
{
    display(obj.Dump());
    display(obj);
};

var readTask = Task.Run(() => source.Process());

Task.Delay(TimeSpan.FromSeconds(1)).Wait();
// source.StopProcessing();
session.Stop();
display("Stopping");
readTask.Wait();
display("Stopped");